In [246]:
import numpy as np
import matplotlib.pyplot as plt
import control as ct
from numpy import array
import scienceplots

np.set_printoptions(precision=3, suppress=True)

plt.style.use([
    'grid',
    'retro'
])

In [247]:
Ms = 1500
Mu = 150

Ks = 30000
Ku = 30000

Cs = 500

M = np.diag([Ms, Mu])

K = np.array([
    [Ks, -Ks],
    [-Ks, Ks+Ku]
])

C = np.array([
    [Cs, -Cs],
    [-Cs, Cs]
])

N = M.shape[0]

In [248]:
_M = -np.linalg.inv(M)

A = np.block([
    [np.zeros((N, N)), np.eye(N)],
    [_M@K, _M@C]
])

F = np.array([
    [0,  0],
    [Ks, Ku],
])

B = np.block([
    [np.zeros((N, N))],
    [-_M@F]
])

_C = np.eye(2*N)
D = np.zeros((2*N, N))

sys = ct.ss(A, B, _C, D)
tf_sys = ct.ss2tf(sys)

In [249]:
B

array([[  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [200., 200.]])

In [254]:
%matplotlib qt

ct.bode(tf_sys[0,0], omega_limits=(1e-1, 1e3))
ct.bode(tf_sys[2,0], omega_limits=(1e-1, 1e3))

plt.legend(['$x_u$', '$x_s$'])

In [255]:
T = np.linspace(0, 500, 1000)
U = np.zeros((2, T.shape[0]))

U[0, :] = np.sin(2*np.pi*.1*T)
U[1, :] = np.sin(2*np.pi*.2*T)

_,y = ct.forced_response(
    sys, T, U
)

plt.plot(T, y[0])
plt.plot(T, y[1])


In [252]:
w     = np.linspace(1e-1, 10, 1000)
zetas = np.linspace(.1, 2, 6)

epsilon = Ms/Mu
ws      = np.sqrt(Ks/Ms)
wu      = np.sqrt(Ku/Mu)
alpha   = ws/wu
r       = w/ws

fig, axs = plt.subplots(3, 1, sharex=True)

# axs[0] = mu: Xs/Y transmissibility suspended mass
# axs[1] = tau: Xu/Y transmissibility unsuspended mass
# axs[2] = eta: Z/Y displacement transmissibility

for zeta in zetas:
    Z1 = (r**2*(r**2*alpha**2-1)+(1-(1+epsilon)*r**2*alpha**2))
    Z2 = 2*zeta*r*(1-(1+epsilon)*r**2*alpha**2)

    mu  = np.sqrt( (4*zeta**2*r**2 + 1)/(Z1**2+Z2**2) )
    tau = np.sqrt( (4*zeta**2*r**2+1+r**2*(r**2-2))/(Z1**2+Z2**2) )
    eta = np.sqrt(r**4 / (Z1**2+Z2**2))

    axs[0].plot(w, mu,  label=f'$\\zeta={zeta:.1f}$')
    axs[1].plot(w, tau, label=f'$\\zeta={zeta:.1f}$')
    axs[2].plot(w, eta, label=f'$\\zeta={zeta:.1f}$')

# put the Y label in diagonal to save space

axs[0].set_ylabel('$\mu = \\frac{X_s}{y}$')
axs[1].set_ylabel('$\\tau = \\frac{X_u}{Y}$')
axs[2].set_ylabel('$\eta = \\frac{Z}{Y}$')
axs[2].set_xlabel('$r$')

plt.legend()

In [253]:
print(f'epsilon = {epsilon:.3f}')
print(f'ws = {ws/2/np.pi:.3f}')
print(f'wu = {wu/2/np.pi:.3f}')
print(f'alpha = {alpha:.3f}')

epsilon = 10.000
ws = 0.712
wu = 2.251
alpha = 0.316
